In [1]:
import re
from json import dump
from tqdm import tqdm
import time
from collections import defaultdict
import urllib.request

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pickle

In [4]:
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 5) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=price-desc&page={page}"
    print(f"Visiting {url}")
    bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent':"PostmanRuntime/7.6.0"})))

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])


Visiting https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page=1
Visiting https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page=2
Visiting https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page=3
Visiting https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page=4


In [6]:
len(url_links)


80

https://www.domain.com.au/rent/?ssubs=0&postcode=3053

In [7]:
url = BASE_URL+"/rent/?ssubs=0&postcode=3053"

In [8]:
bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent':"PostmanRuntime/7.6.0"})))

In [18]:
int(bs_object.find_all("h1", {"class": "css-ekkwk0"})[0].text.split(" ")[0])

168

In [32]:
count_dict = {'postcode': [], 'count': []}

In [33]:
for postcode in tqdm(range(3000, 3996)):
    url = BASE_URL+f"/rent/?ssubs=0&postcode={postcode}"
    # print(f"Visiting postcode {postcode}", end=" ",flush=True)
    bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent':"PostmanRuntime/7.6.0"})))
    count = int(bs_object.find_all("h1", {"class": "css-ekkwk0"})[0].text.split(" ")[0])
    count_dict['postcode'].append(postcode)
    count_dict['count'].append(count)
    # print(f"Count: {count}", end="\r")
    time.sleep(1)

100%|██████████| 996/996 [24:03<00:00,  1.45s/it]


In [31]:
count_dict["postcode"]

[]

In [27]:
len(count_dict["count"])

117

In [22]:
for i in range(10):
    print(f"Visiting postcode {i}",flush=True)
    time.sleep(1)
    print("Done", end="\r")

Visiting postcode 0
Visiting postcode 1
Visiting postcode 2
Visiting postcode 3


KeyboardInterrupt: 

In [34]:
import json
out_dir = "../data/raw/"
with open(out_dir+"postcode_counts.json", "w") as f:
    json.dump(count_dict, f)

In [1]:
import pandas as pd
import folium
import geopandas as gpd

postcode_zones = gpd.read_file("../data/landing/postcode_zone/POSTCODE_POLYGON.shp").to_crs(epsg=4326)
postcode_count = pd.read_json("../data/raw/postcode_counts.json")

In [ ]:
m = folium.Map(location=[-37.8, 144.97],zoom_start=8)
postcode_zones = gpd.read_file("../data/landing/postcode_zone/POSTCODE_POLYGON.shp").to_crs(epsg=4326)
geoJSON = postcode_zones[['POSTCODE', 'geometry']].drop_duplicates('POSTCODE').to_json()

folium.Choropleth(
    geo_data=geoJSON,   
    name='choropleth',
    data=postcode_count,
    columns=["postcode","count"],
    key_on="properties.POSTCODE",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Count of rental properties"
).add_to(m)
m
# m.save(f"{PLOT_DIR}arev_effi_zone.html")

## scrape

In [4]:
onthehouse_url = "https://www.onthehouse.com.au/property/vic/carlton-3053/1301-6-leicester-st-carlton-vic-3053-18673607"

In [5]:
bs_object = BeautifulSoup(urlopen(Request(onthehouse_url, headers={'User-Agent':"PostmanRuntime/7.6.0"})))

In [17]:
import re
bs_object.findAll(string=re.compile("511,800"))

['Sold on 13 Jun 2024 for $511,800', '$511,800', 'Sold for $511,800']

In [16]:
type(k)

bs4.element.ResultSet

# random code


In [1]:
import requests

payload = { 'api_key': '5b3331933de07d890c95c21bd09a52cd', 'url': 'https://www.oldlistings.com.au/real-estate/VIC/Bentleigh+East/3165/rent/110', 'follow_redirect': 'false' }
r = requests.get('https://api.scraperapi.com/', params=payload)
print(r.text)


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8" />
<meta name="theme-color" content="#3AB54A" />
<meta http-equiv="x-ua-compatible" content="ie=edge" />
<meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" />
<link rel="next" href="/real-estate/VIC/Bentleigh+East/3165/rent/111" />
<link rel="prev" href="/real-estate/VIC/Bentleigh+East/3165/rent/109" />
<meta name="base.search" content="/real-estate/VIC/Bentleigh+East/3165/rent/" />
<meta name="ROBOTS" content="ALL" />
<meta name="country" content="Australia" />
<meta name="geo.position" content="-37.918170;145.057670" />
<meta name="geo.region" content="VIC" />
<meta name="geo.placename" content="BENTLEIGH EAST" />
<meta name="description" content="Bentleigh East Rental Real Estate Historical Prices Page 110, Old Listings with recorded property information" />
<meta name="keywords" content="Bentleigh East real estate rental listings Page 110" />
<script src="//ajax.googleap

# Preprocess

In [1]:
import bs4
from tqdm import tqdm
import sys
import gc
import psutil
import json
process = psutil.Process()
sys.path.append("../")
from scripts.oldlistings import get_oldlistings_page
import os
test_path = "../data/landing/oldlistings/rent_html/3000_Melbourne_6.html"

In [2]:

listings = []
count = 0
directory = '../data/landing/oldlistings/rent_html'
import os
all = os.listdir(directory)
l1 = all[0:6000]
l2 = all[6000:12000]
l3 = all[12000:18000]
l4 = all[18000:24000]
l5 = all[24000:]


In [3]:
for filename in tqdm(l5):
    file_path = os.path.join(directory, filename)
    count+=1
    if count > 200:
        gc.collect()
        count = 0
        # print(process.memory_info().rss)
    if os.path.isfile(file_path):
        with open(file_path,'r') as f:
            page = bs4.BeautifulSoup(f.read(), "html.parser")
            _l = get_oldlistings_page(page)
            page.decompose()
        listings += _l
        # Add code to process each file
    else:
        print(f"{file_path} is not a file")


out_dir = "../data/raw/"
json.dump(listings, open(out_dir+"oldlistings_rent_5.json", "w"))

  0%|          | 0/5282 [00:00<?, ?it/s]

100%|██████████| 5282/5282 [08:18<00:00, 10.60it/s]


In [3]:
len(all)

29282

In [3]:
import json
out_dir = "../data/raw/"
json.dump(listings, open(out_dir+"oldlistings_rent.json", "w"))

In [3]:
len(listings)

494485

In [5]:
## read json
import json
out_dir = "../data/raw/"
listings = json.load(open(out_dir+"oldlistings_rent.json", "r"))

In [9]:
count = 0
for l in listings:
    count+=len(l['rented_prices'])

In [10]:
count

0

In [7]:
listings[0]['rented_prices']

[]

In [4]:
page = bs4.BeautifulSoup(open(test_path,'r').read(), "html.parser")
get_oldlistings_page(page)

[defaultdict(None,
             {'lat': '-37.8096700',
              'lng': '144.9566640',
              'rented_prices': [{'date': 'August 2024',
                'price': '$60 per week'},
               {'date': 'July 2023', 'price': '$70 per week'}],
              'address': 'CPB/243-263 FRANKLIN STREET, MELBOURNE',
              'meta_data': [{'label': 'car',
                'description': 'Car',
                'quantity': '1'},
               {'label': 'type',
                'description': 'Category',
                'quantity': 'Unit/apmt'}]}),
 defaultdict(None,
             {'lat': '-37.809227',
              'lng': '144.968484',
              'rented_prices': [{'date': 'August 2024',
                'price': '$70 per week'}],
              'address': 'CPA/8 EXPLORATION LANE, MELBOURNE',
              'meta_data': [{'label': 'car',
                'description': 'Car',
                'quantity': '1'},
               {'label': 'type',
                'description': 'Category',

In [1]:
import pandas as pd
import geopandas as gpd

In [6]:
rental_df = pd.read_csv('../data/curated/rental-17-24.csv')
suburb_shape = gpd.read_file('../data/curated/shapefiles/SAL_2021_AUST_GDA2020_SHP/SAL_2021_AUST_GDA2020.shp')
suburb_shape = suburb_shape[['SAL_CODE21', 'SAL_NAME21' ,'geometry']]

In [7]:
rental_gdf = gpd.GeoDataFrame(rental_df, geometry=gpd.points_from_xy(rental_df.lng, rental_df.lat), crs='EPSG:7844')
rental_gdf = gpd.sjoin(rental_gdf, suburb_shape, how='left', predicate='within')\
    .drop(columns=['index_right'])

In [11]:
rental_gdf

,address,suburb,bed,bath,car,type,year,sa2_code,median_income,population,...,unemployment_rate,lat,lng,geometry,rented_price,city_index,time_city,avg_property_price,SAL_CODE21,SAL_NAME21
0,"201/560 LONSDALE STREET, MELBOURNE",MELBOURNE,2.0,2.0,1.0,Unit/apmt,2023,206041505.0,44492.400500,20027.0,...,3.691667,-37.813730,144.955580,POINT (144.95558 -37.81373),800,0,263.3,682488.770000,21640,Melbourne
1,"201/560 LONSDALE STREET, MELBOURNE",MELBOURNE,2.0,2.0,1.0,Unit/apmt,2023,206041505.0,44492.400500,20027.0,...,3.691667,-37.813730,144.955580,POINT (144.95558 -37.81373),800,0,263.3,682488.770000,21640,Melbourne
2,"201/560 LONSDALE STREET, MELBOURNE",MELBOURNE,2.0,2.0,1.0,Unit/apmt,2021,206041505.0,39300.000000,16098.0,...,4.200000,-37.813730,144.955580,POINT (144.95558 -37.81373),540,0,263.3,619543.745192,21640,Melbourne
3,"1702/560 LONSDALE STREET, MELBOURNE",MELBOURNE,2.0,1.0,0.0,Unit/apmt,2023,206041505.0,44492.400500,20027.0,...,3.691667,-37.813730,144.955580,POINT (144.95558 -37.81373),720,0,263.3,682488.770000,21640,Melbourne
4,"1702/560 LONSDALE STREET, MELBOURNE",MELBOURNE,2.0,1.0,0.0,Unit/apmt,2023,206041505.0,44492.400500,20027.0,...,3.691667,-37.813730,144.955580,POINT (144.95558 -37.81373),650,0,263.3,682488.770000,21640,Melbourne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409722,"7 ROSSETTI COURT, PAKENHAM",PAKENHAM,3.0,1.0,0.0,House,2024,212011552.0,55683.420715,28466.0,...,4.014286,-38.069069,145.463520,POINT (145.46352 -38.06907),520,0,3251.4,843219.858156,22027,Pakenham
1409723,"7 DISCOVERY STREET, PAKENHAM",PAKENHAM,3.0,2.0,2.0,House,2024,212011551.0,57772.670788,7044.0,...,4.014286,-38.078518,145.482727,POINT (145.48273 -38.07852),500,0,3126.4,843219.858156,22027,Pakenham
1409724,"7 DISCOVERY STREET, PAKENHAM",PAKENHAM,3.0,2.0,2.0,House,2024,212011551.0,57772.670788,7044.0,...,4.014286,-38.078518,145.482727,POINT (145.48273 -38.07852),500,0,3126.4,843219.858156,22027,Pakenham
1409725,"19 PORTILLO CRESCENT, PAKENHAM",PAKENHAM,4.0,2.0,2.0,Unit/apmt,2024,212011550.0,58924.808098,9820.0,...,4.014286,-38.052810,145.454962,POINT (145.45496 -38.05281),580,0,3297.1,843219.858156,22027,Pakenham


In [16]:
rental_gdf.drop('suburb',axis=1,inplace=True)
rental_gdf.rename(columns={
    'SAL_CODE21':'suburb_id',
    'SAL_NAME21':'suburb'
}, inplace=True)

In [18]:
rental_gdf.to_csv('../data/curated/data17-24.csv')